# Texterfassung Steuerzentrale für Yijing

## 1. Imports und Konfiguration

In [ ]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any

import google.generativeai as genai
import pandas as pd
from IPython.display import Markdown, display

# Setze das Arbeitsverzeichnis auf das Projektverzeichnis
os.chdir("../")  # Passe den Pfad entsprechend an
print(os.getcwd())

from jupyter_json_viewer import display_json

### API-Konfiguration

In [ ]:
# Konfiguration des API-Schlüssels
api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("Die Umgebungsvariable 'API_KEY' ist nicht gesetzt.")
genai.configure(api_key=api_key)

### Modellwahl

In [ ]:
# Auswahl des Modells
model_speed = {
    'dumb': 'gemini-1.5-flash-8b',
    'fast': 'gemini-1.5-flash-latest',
    'clever': 'gemini-1.5-pro-latest',
    'experimental': 'gemini-exp-1121',
}
model_type = os.getenv("MODEL_TYPE", model_speed['fast'])
print("Verwendetes Modell:", model_type)

## 2. Datenimport

##### hexagram_data

In [ ]:
path = Path("yijing/resources/hexagram_json/")
json_files = {}

files = os.listdir(path)
for file in files:
    if file.endswith(".json"):
        json_files[Path(file).stem] = file

hexagram_data = []

for key, file in json_files.items():
    print(f"{path}/{file}")
    with open(f"{path}/{file}") as f:
        # Check if the file is not empty
        content = f.read()
        if content:
            try:
                data = json.loads(content)
                hexagram_data.append(data)
            except json.JSONDecodeError as e:
                print(f"Error: {file} could not be loaded.")
                print(e)
        else:
            print(f"Warning: {file} is empty and will be skipped.")

print("Hexagramm-Daten geladen:", len(hexagram_data))


In [ ]:
def load_hexagram_data(folder: Path) -> List[Dict[str, Any]]:
    """
    Load hexagram data from JSON files in the specified folder.

    This function scans the given folder for JSON files, reads their content,
    and loads the data into a list of dictionaries. It skips empty files and
    handles JSON decoding errors gracefully.

    Args:
        folder (Path): The path to the folder containing the JSON files.

    Returns:
        List[Dict[str, Any]]: A list of dictionaries containing the hexagram data.

    Raises:
        FileNotFoundError: If the specified folder does not exist.
        IOError: If there is an error reading a file.
        json.JSONDecodeError: If there is an error decoding a JSON file.

    Example:
        >>> from pathlib import Path
        >>> hexagram_data = load_hexagram_data(Path('/path/to/folder'))
        >>> print(hexagram_data)
    """
    json_files = {}
    files = os.listdir(folder)
    for file in files:
        if file.endswith(".json"):
            json_files[Path(file).stem] = file

    hexagram_data = []

    for key, file in json_files.items():
        #print(f"{folder}/{file}")
        with open(f"{folder}/{file}") as f:
            # Check if the file is not empty
            content = f.read()
            if content:
                try:
                    data = json.loads(content)
                    hexagram_data.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error: {file} could not be loaded.")
                    print(e)
            else:
                print(f"Warning: {file} is empty and will be skipped.")

    print("Hexagramm-Daten geladen:", len(hexagram_data))
    return hexagram_data

# Usage
hexagram_data = load_hexagram_data(path)

In [ ]:
display_json(hexagram_data,
             title="Einfache JSON-Daten",
             dark_mode=True,
             collapsed=True
             )

In [ ]:
import json

# Beispiel für verschachtelte JSON-Daten
data = hexagram_data[10]

# Verschachtelte JSON-Daten als formatierten String anzeigen
print(json.dumps(data, indent=4)[:1000])


In [ ]:
from IPython.display import display, HTML
import json

# Beispiel für verschachtelte JSON-Daten
data = hexagram_data[10]


# JSON-Daten als HTML darstellen
formatted_json = json.dumps(data, indent=4).replace(" ", "&nbsp;").replace("\n", "<br>")
display(HTML(f"<pre>{formatted_json}</pre>"))


In [ ]:
text = "en. F\u00f6rdernd ist Beharrlichkeit. Ein M\u00e4dchen nehmen bringt Heil."
decoded_text = text.encode('iso-8859-1').decode('unicode_escape')
print(decoded_text)


# JSON Viewer with Dark Mode

In [ ]:
import json
from IPython.display import HTML, display
from typing import Any, Optional

def display_json(data: Any, 
                title: Optional[str] = None,
                max_depth: int = None,
                collapsed: bool = False,
                indent_size: int = 24,
                dark_mode: bool = False) -> None:
    """
    Zeigt JSON-Daten in Jupyter Notebooks mit verbesserter visueller Hierarchie an.
    
    Args:
        data: Die JSON-Daten zum Anzeigen
        title: Optionaler Titel über der JSON-Ansicht
        max_depth: Maximale Verschachtelungstiefe zum Anzeigen
        collapsed: Ob die JSON-Ansicht initial eingeklappt sein soll
        indent_size: Größe der Einrückung in Pixeln
        dark_mode: Aktiviert das dunkle Farbschema
    """
    
    # Light/Dark Mode Farbschemata
    colors = {
        'light': {
            'background': '#f8f9fa',
            'text': '#2c3e50',
            'string': '#28a745',
            'number': '#0066cc',
            'boolean': '#e83e8c',
            'null': '#6c757d',
            'key': '#2c3e50',
            'line': '#dee2e6',
            'collapsible_bg': '#e9ecef',
            'collapsible_hover': '#dee2e6',
            'collapsible_border': '#ced4da',
            'property_hover': 'rgba(0,0,0,0.02)',
            'shadow': 'rgba(0,0,0,0.05)'
        },
        'dark': {
            'background': '#1e1e1e',
            'text': '#d4d4d4',
            'string': '#6A9955',
            'number': '#569CD6',
            'boolean': '#C586C0',
            'null': '#808080',
            'key': '#4EC9B0',  # Geändert von #9CDCFE zu einem angenehmen Grünton
            'line': '#404040',
            'collapsible_bg': '#2d2d2d',
            'collapsible_hover': '#383838',
            'collapsible_border': '#404040',
            'property_hover': 'rgba(255,255,255,0.02)',
            'shadow': 'rgba(0,0,0,0.2)'
        }
    }
    
    theme = colors['dark'] if dark_mode else colors['light']
    
    styles = f"""
    <style>
        .json-viewer {{
            font-family: 'JetBrains Mono', 'Fira Code', Consolas, monospace;
            font-size: 10px;
            background-color: {theme['background']};
            color: {theme['text']};
            border-radius: 8px;
            padding: 1.5em;
            line-height: 1.6;
            box-shadow: 0 2px 8px {theme['shadow']};
        }}
        .json-title {{
            font-size: 12px;
            font-weight: bold;
            margin-bottom: 15px;
            color: {theme['text']};
            border-bottom: 2px solid {theme['line']};
            padding-bottom: 8px;
        }}
        .json-string {{ 
            color: {theme['string']}; 
            word-break: break-word;
        }}
        .json-number {{ color: {theme['number']}; }}
        .json-boolean {{ color: {theme['boolean']}; }}
        .json-null {{ color: {theme['null']}; }}
        .json-key {{ 
            color: {theme['key']}; 
            font-weight: 600;
            margin-right: 8px;
        }}
        .json-bracket {{ 
            color: {theme['text']};
            opacity: 0.7;
        }}
        .json-container {{
            position: relative;
        }}
        .vertical-line {{
            position: absolute;
            left: 8px;
            top: 0;
            bottom: 0;
            width: 1px;
            background-color: {theme['line']};
        }}
        .collapsible {{
            cursor: pointer;
            padding: 2px 8px;
            background-color: {theme['collapsible_bg']};
            border-radius: 4px;
            display: inline-block;
            margin: 2px;
            transition: all 0.2s;
            border: 1px solid transparent;
        }}
        .collapsible:hover {{
            background-color: {theme['collapsible_hover']};
            border-color: {theme['collapsible_border']};
        }}
        .content {{
            display: block;
            position: relative;
            margin-left: 12px;
        }}
        .collapsed {{
            display: none;
        }}
        .property {{
            display: flex;
            align-items: flex-start;
            padding: 2px 0;
            border-radius: 4px;
        }}
        .property:hover {{
            background-color: {theme['property_hover']};
        }}
        .key-value-separator {{
            margin: 0 8px;
            color: {theme['null']};
        }}
        .depth-marker {{
            color: {theme['null']};
            margin-right: 8px;
            font-size: 10px;
            opacity: 0.5;
        }}
    </style>
    """
    
    script = """
    <script>
        function toggleCollapse(element) {
            const content = element.nextElementSibling;
            content.classList.toggle('collapsed');
            if (content.classList.contains('collapsed')) {
                element.textContent = element.textContent.replace('▼', '▶');
            } else {
                element.textContent = element.textContent.replace('▶', '▼');
            }
        }
    </script>
    """
    
    def format_value(v: Any, depth: int = 0, path: str = '') -> str:
        if max_depth is not None and depth >= max_depth:
            return '<span class="json-string">"..."</span>'
        
        indent = depth * indent_size
        
        if isinstance(v, dict):
            if not v:
                return '<span class="json-bracket">{}</span>'
                
            collapse_class = ' collapsed' if collapsed else ''
            result = [
                f'<div class="collapsible" onclick="toggleCollapse(this)">{"▶" if collapsed else "▼"}</div>',
                f'<div class="content{collapse_class}"><div class="json-container">'
            ]
            
            for k, val in v.items():
                current_path = f"{path}.{k}" if path else k
                result.append(
                    f'<div class="property">'
                    f'<span class="depth-marker">{"┌" if k == list(v.keys())[0] else "└" if k == list(v.keys())[-1] else "├"}</span>'
                    f'<span class="json-key">"{k}"</span>'
                    f'<span class="key-value-separator">:</span>'
                    f'{format_value(val, depth + 1, current_path)}'
                    f'</div>'
                )
            
            result.append('</div></div>')
            return '\n'.join(result)
            
        elif isinstance(v, list):
            if not v:
                return '<span class="json-bracket">[]</span>'
                
            collapse_class = ' collapsed' if collapsed else ''
            result = [
                f'<div class="collapsible" onclick="toggleCollapse(this)">{"▶" if collapsed else "▼"}</div>',
                f'<div class="content{collapse_class}"><div class="json-container">'
            ]
            
            for i, item in enumerate(v):
                current_path = f"{path}[{i}]"
                result.append(
                    f'<div class="property">'
                    f'<span class="depth-marker">{"┌" if i == 0 else "└" if i == len(v)-1 else "├"}</span>'
                    f'{format_value(item, depth + 1, current_path)}'
                    f'</div>'
                )
            
            result.append('</div></div>')
            return '\n'.join(result)
            
        elif isinstance(v, str):
            return f'<span class="json-string">"{v}"</span>'
        elif isinstance(v, bool):
            return f'<span class="json-boolean">{str(v).lower()}</span>'
        elif v is None:
            return f'<span class="json-null">null</span>'
        else:
            return f'<span class="json-number">{v}</span>'
    
    html_content = styles + script
    if title:
        html_content += f'<div class="json-title">{title}</div>'
    
    html_content += f'<div class="json-viewer">{format_value(data)}</div>'
    
    display(HTML(html_content))

In [ ]:
# Dark Mode aktivieren
display_json(data, title="Meine JSON-Daten", dark_mode=True)

# Light Mode (Standard)
#display_json(data, title="Meine JSON-Daten", dark_mode=False)

In [ ]:
import json
from IPython.display import HTML, display
from typing import Any, Optional

def display_json(data: Any, 
                title: Optional[str] = None,
                max_depth: int = None,
                collapsed: bool = False,
                indent_size: int = 24,
                dark_mode: bool = False) -> None:
    """
    Zeigt JSON-Daten in Jupyter Notebooks mit verbesserter visueller Hierarchie an.
    
    Args:
        data: Die JSON-Daten zum Anzeigen
        title: Optionaler Titel über der JSON-Ansicht
        max_depth: Maximale Verschachtelungstiefe zum Anzeigen
        collapsed: Ob die JSON-Ansicht initial eingeklappt sein soll
        indent_size: Größe der Einrückung in Pixeln
        dark_mode: Aktiviert das dunkle Farbschema
    """
    
    # Light/Dark Mode Farbschemata
    colors = {
        'light': {
            'background': '#f8f9fa',
            'text': '#2c3e50',
            'string': '#28a745',
            'number': '#0066cc',
            'boolean': '#e83e8c',
            'null': '#6c757d',
            'key': '#2c3e50',
            'line': '#dee2e6',
            'collapsible_bg': '#e9ecef',
            'collapsible_hover': '#dee2e6',
            'collapsible_border': '#ced4da',
            'property_hover': 'rgba(0,0,0,0.02)',
            'shadow': 'rgba(0,0,0,0.05)'
        },
        'dark': {
            'background': '#1e1e1e',
            'text': '#d4d4d4',
            'string': '#6A9955',
            'number': '#569CD6',
            'boolean': '#C586C0',
            'null': '#808080',
            'key': '#9CDCFE',
            'line': '#404040',
            'collapsible_bg': '#2d2d2d',
            'collapsible_hover': '#383838',
            'collapsible_border': '#404040',
            'property_hover': 'rgba(255,255,255,0.02)',
            'shadow': 'rgba(0,0,0,0.2)'
        }
    }
    
    theme = colors['dark'] if dark_mode else colors['light']
    
    styles = f"""
    <style>
        .json-viewer {{
            font-family: 'JetBrains Mono', 'Fira Code', Consolas, monospace;
            font-size: 10px;
            background-color: {theme['background']};
            color: {theme['text']};
            border-radius: 8px;
            padding: 1.5em;
            line-height: 1.6;
            box-shadow: 0 2px 8px {theme['shadow']};
        }}
        .json-title {{
            font-size: 12px;
            font-weight: bold;
            margin-bottom: 15px;
            color: {theme['text']};
            border-bottom: 2px solid {theme['line']};
            padding-bottom: 8px;
        }}
        .json-string {{ 
            color: {theme['string']}; 
            word-break: break-word;
        }}
        .json-number {{ color: {theme['number']}; }}
        .json-boolean {{ color: {theme['boolean']}; }}
        .json-null {{ color: {theme['null']}; }}
        .json-key {{ 
            color: {theme['key']}; 
            font-weight: 600;
            margin-right: 8px;
        }}
        .json-bracket {{ 
            color: {theme['text']};
            opacity: 0.7;
        }}
        .json-container {{
            position: relative;
        }}
        .vertical-line {{
            position: absolute;
            left: 8px;
            top: 0;
            bottom: 0;
            width: 1px;
            background-color: {theme['line']};
        }}
        .collapsible {{
            cursor: pointer;
            padding: 2px 8px;
            background-color: {theme['collapsible_bg']};
            border-radius: 4px;
            display: inline-block;
            margin: 2px;
            transition: all 0.2s;
            border: 1px solid transparent;
        }}
        .collapsible:hover {{
            background-color: {theme['collapsible_hover']};
            border-color: {theme['collapsible_border']};
        }}
        .content {{
            display: block;
            position: relative;
            margin-left: 12px;
        }}
        .collapsed {{
            display: none;
        }}
        .property {{
            display: flex;
            align-items: flex-start;
            padding: 2px 0;
            border-radius: 4px;
        }}
        .property:hover {{
            background-color: {theme['property_hover']};
        }}
        .key-value-separator {{
            margin: 0 8px;
            color: {theme['null']};
        }}
        .depth-marker {{
            color: {theme['null']};
            margin-right: 8px;
            font-size: 10px;
            opacity: 0.5;
        }}
    </style>
    """
    
    script = """
    <script>
        function toggleCollapse(element) {
            const content = element.nextElementSibling;
            content.classList.toggle('collapsed');
            if (content.classList.contains('collapsed')) {
                element.textContent = element.textContent.replace('▼', '▶');
            } else {
                element.textContent = element.textContent.replace('▶', '▼');
            }
        }
    </script>
    """
    
    def format_value(v: Any, depth: int = 0, path: str = '') -> str:
        if max_depth is not None and depth >= max_depth:
            return '<span class="json-string">"..."</span>'
        
        indent = depth * indent_size
        
        if isinstance(v, dict):
            if not v:
                return '<span class="json-bracket">{}</span>'
                
            collapse_class = ' collapsed' if collapsed else ''
            result = [
                f'<div class="collapsible" onclick="toggleCollapse(this)">{"▶" if collapsed else "▼"}</div>',
                f'<div class="content{collapse_class}"><div class="json-container">'
            ]
            
            for k, val in v.items():
                current_path = f"{path}.{k}" if path else k
                result.append(
                    f'<div class="property">'
                    f'<span class="depth-marker">{"┌" if k == list(v.keys())[0] else "└" if k == list(v.keys())[-1] else "├"}</span>'
                    f'<span class="json-key">"{k}"</span>'
                    f'<span class="key-value-separator">:</span>'
                    f'{format_value(val, depth + 1, current_path)}'
                    f'</div>'
                )
            
            result.append('</div></div>')
            return '\n'.join(result)
            
        elif isinstance(v, list):
            if not v:
                return '<span class="json-bracket">[]</span>'
                
            collapse_class = ' collapsed' if collapsed else ''
            result = [
                f'<div class="collapsible" onclick="toggleCollapse(this)">{"▶" if collapsed else "▼"}</div>',
                f'<div class="content{collapse_class}"><div class="json-container">'
            ]
            
            for i, item in enumerate(v):
                current_path = f"{path}[{i}]"
                result.append(
                    f'<div class="property">'
                    f'<span class="depth-marker">{"┌" if i == 0 else "└" if i == len(v)-1 else "├"}</span>'
                    f'{format_value(item, depth + 1, current_path)}'
                    f'</div>'
                )
            
            result.append('</div></div>')
            return '\n'.join(result)
            
        elif isinstance(v, str):
            return f'<span class="json-string">"{v}"</span>'
        elif isinstance(v, bool):
            return f'<span class="json-boolean">{str(v).lower()}</span>'
        elif v is None:
            return f'<span class="json-null">null</span>'
        else:
            return f'<span class="json-number">{v}</span>'
    
    html_content = styles + script
    if title:
        html_content += f'<div class="json-title">{title}</div>'
    
    html_content += f'<div class="json-viewer">{format_value(data)}</div>'
    
    display(HTML(html_content))

In [ ]:
# Dark Mode aktivieren
display_json(data, title="Meine JSON-Daten", dark_mode=True)

# Light Mode (Standard)
#display_json(data, title="Meine JSON-Daten", dark_mode=False)

In [ ]:
import json
from IPython.display import HTML, display
from typing import Any, Optional

def display_json(data: Any, 
                title: Optional[str] = None,
                max_depth: int = None,
                collapsed: bool = False,
                indent_size: int = 24) -> None:
    """
    Zeigt JSON-Daten in Jupyter Notebooks mit verbesserter visueller Hierarchie an.
    
    Args:
        data: Die JSON-Daten zum Anzeigen
        title: Optionaler Titel über der JSON-Ansicht
        max_depth: Maximale Verschachtelungstiefe zum Anzeigen
        collapsed: Ob die JSON-Ansicht initial eingeklappt sein soll
        indent_size: Größe der Einrückung in Pixeln
    """
    
    styles = """
    <style>
        .json-viewer {
            font-family: 'JetBrains Mono', 'Fira Code', Consolas, monospace;
            font-size: 10px;
            background-color: #f8f9fa;
            border-radius: 8px;
            padding: 1.5em;
            line-height: 1.6;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }
        .json-title {
            font-size: 12px;
            font-weight: bold;
            margin-bottom: 15px;
            color: #2c3e50;
            border-bottom: 2px solid #e9ecef;
            padding-bottom: 8px;
        }
        .json-string { 
            color: #28a745; 
            word-break: break-word;
        }
        .json-number { color: #0066cc; }
        .json-boolean { color: #e83e8c; }
        .json-null { color: #6c757d; }
        .json-key { 
            color: #2c3e50; 
            font-weight: 600;
            margin-right: 8px;
        }
        .json-bracket { 
            color: #2c3e50;
            opacity: 0.7;
        }
        .json-container {
            position: relative;
        }
        .vertical-line {
            position: absolute;
            left: 8px;
            top: 0;
            bottom: 0;
            width: 1px;
            background-color: #dee2e6;
        }
        .collapsible {
            cursor: pointer;
            padding: 2px 8px;
            background-color: #e9ecef;
            border-radius: 4px;
            display: inline-block;
            margin: 2px;
            transition: all 0.2s;
            border: 1px solid transparent;
        }
        .collapsible:hover {
            background-color: #dee2e6;
            border-color: #ced4da;
        }
        .content {
            display: block;
            position: relative;
            margin-left: 12px;
        }
        .collapsed {
            display: none;
        }
        .property {
            display: flex;
            align-items: flex-start;
            padding: 2px 0;
        }
        .property:hover {
            background-color: rgba(0,0,0,0.02);
        }
        .key-value-separator {
            margin: 0 8px;
            color: #6c757d;
        }
        .depth-marker {
            color: #adb5bd;
            margin-right: 8px;
            font-size: 10px;
            opacity: 0.5;
        }
    </style>
    """
    
    script = """
    <script>
        function toggleCollapse(element) {
            const content = element.nextElementSibling;
            content.classList.toggle('collapsed');
            if (content.classList.contains('collapsed')) {
                element.textContent = element.textContent.replace('▼', '▶');
            } else {
                element.textContent = element.textContent.replace('▶', '▼');
            }
        }
    </script>
    """
    
    def format_value(v: Any, depth: int = 0, path: str = '') -> str:
        if max_depth is not None and depth >= max_depth:
            return '<span class="json-string">"..."</span>'
        
        indent = depth * indent_size
        
        if isinstance(v, dict):
            if not v:
                return '<span class="json-bracket">{}</span>'
                
            collapse_class = ' collapsed' if collapsed else ''
            result = [
                f'<div class="collapsible" onclick="toggleCollapse(this)">{"▶" if collapsed else "▼"}</div>',
                f'<div class="content{collapse_class}"><div class="json-container">'
            ]
            
            for k, val in v.items():
                current_path = f"{path}.{k}" if path else k
                result.append(
                    f'<div class="property">'
                    f'<span class="depth-marker">{"┌" if k == list(v.keys())[0] else "└" if k == list(v.keys())[-1] else "├"}</span>'
                    f'<span class="json-key">"{k}"</span>'
                    f'<span class="key-value-separator">:</span>'
                    f'{format_value(val, depth + 1, current_path)}'
                    f'</div>'
                )
            
            result.append('</div></div>')
            return '\n'.join(result)
            
        elif isinstance(v, list):
            if not v:
                return '<span class="json-bracket">[]</span>'
                
            collapse_class = ' collapsed' if collapsed else ''
            result = [
                f'<div class="collapsible" onclick="toggleCollapse(this)">{"▶" if collapsed else "▼"}</div>',
                f'<div class="content{collapse_class}"><div class="json-container">'
            ]
            
            for i, item in enumerate(v):
                current_path = f"{path}[{i}]"
                result.append(
                    f'<div class="property">'
                    f'<span class="depth-marker">{"┌" if i == 0 else "└" if i == len(v)-1 else "├"}</span>'
                    f'{format_value(item, depth + 1, current_path)}'
                    f'</div>'
                )
            
            result.append('</div></div>')
            return '\n'.join(result)
            
        elif isinstance(v, str):
            return f'<span class="json-string">"{v}"</span>'
        elif isinstance(v, bool):
            return f'<span class="json-boolean">{str(v).lower()}</span>'
        elif v is None:
            return f'<span class="json-null">null</span>'
        else:
            return f'<span class="json-number">{v}</span>'
    
    html_content = styles + script
    if title:
        html_content += f'<div class="json-title">{title}</div>'
    
    html_content += f'<div class="json-viewer">{format_value(data)}</div>'
    
    display(HTML(html_content))

In [ ]:
data = hexagram_data[10]
data

In [ ]:
# Beispieldaten
data = hexagram_data[10]

# Einfache Nutzung
display_json(data)

# Mit allen Optionen
display_json(data, 
            title="Meine JSON-Daten",
            max_depth=2,  # Begrenzt die Anzeigetiefe
            collapsed=True)  # Initial eingeklappt

##### hexagram_data_df

In [ ]:
df = pd.DataFrame(hexagram_data).copy()
df['hexagram'] = df['hexagram'].astype(str)

# extract "'name': '(.*)'" fom colum 'hexagram' fo column link
df['link'] = df['hexagram'].str.extract(r"'name': '.* / (.*?)'")
df['link'] = df['link'].str.lower()
hexagram_data_df = df
hexagram_data_df.head()

In [ ]:
for col in hexagram_data_df.columns:
    display(Markdown(f"### {col}"))
    display(hexagram_data_df.loc[0, col])

In [ ]:
df = hexagram_data_df.copy()
df = (df['lines'].apply(pd.Series))
df = (df[0].apply(pd.Series))
df

##### linkage_df

In [ ]:
# open import/hexagram_linkage.csv in pandas
linkage_df = (pd.read_csv('import/hexagram_linkage.csv', sep=';')
              .replace('ß', 'ss', regex=True)
              .replace('das durchbeissen', 'das durchbeißen', regex=True)
            )
linkage_df['Alter Name (Deutsch)'] = linkage_df['Alter Name (Deutsch)'].str.lower()
linkage_df

##### hexagramme_df

In [ ]:
# open import/hexagramme.csv in pandas
hexagramme_df = pd.read_csv('import/hexagramme.csv')
hexagramme_df#.iloc[9,:]



## 3. Datenverarbeitung

##### linkage_df

In [ ]:
linkage_csv = linkage_df
hexagram_info_csv = hexagramme_df

# JSON-Daten in DataFrame umwandeln
hexagram_df = hexagram_data_df
hexagram_df

In [ ]:
# Zuordnungen basierend auf "Nummer" oder "Name" verbinden
# Beispiel: Mapping über "Nummer"
result = pd.merge(hexagram_df,
                  linkage_csv,
                  left_on="link",
                  right_on="Vollständiger Name (Deutsch)",
                  how="outer",
                  indicator=True
                  )

result


In [ ]:
# Optionale weitere Datenintegration
result = pd.merge(result, hexagram_info_csv, left_on="Nummer", right_on="Nummer", how="left")

# Ergebnis speichern
#result.to_csv("merged_hexagram_data.csv", index=False, encoding="utf-8")
result

In [ ]:
import pandas as pd
import json

hexagram_df = pd.DataFrame(hexagram_data)

linkage_csv = linkage_df
hexagram_info_csv = hexagramme_df

# JSON-Daten in DataFrame umwandeln
hexagram_list = []
for hex_key, hex_values in hexagram_data.items():
    hex_values["Hexagram_Key"] = hex_key  # Optional: Schlüssel hinzufügen
    hexagram_list.append(hex_values)

hexagram_final_df = pd.DataFrame(hexagram_list)
hexagram_final_df

In [ ]:

# Zuordnungen basierend auf "Nummer" oder "Name" verbinden
# Beispiel: Mapping über "Nummer"
result = pd.merge(hexagram_final_df, linkage_csv, left_on="Hexagram_Key", right_on="Nummer", how="left")

# Optionale weitere Datenintegration
result = pd.merge(result, hexagram_info_csv, on="Nummer", how="left")

# Ergebnis speichern
result.to_csv("merged_hexagram_data.csv", index=False, encoding="utf-8")


##### df_split

In [ ]:
#df = pd.DataFrame(hexagram_data)
df = hexagram_data_df.copy()

#df = (df['hexagram'].apply(pd.Series)
#            .replace('haftende', 'haftene', regex=True)
#            .replace('das durchbeissen', 'das durchbeißen', regex=True)
#            )
df

In [ ]:

df_split['Alter Name (Deutsch)'] = df_split['name'].str.split(' / ').str[1].str.lower()

df_split = (df_split
            .replace('haftende', 'haftene', regex=True)
            .replace('das durchbeißen', 'das durchbeissen', regex=True)
            .replace('das haftene', 'das haftende', regex=True)
            .replace('die wiederkehrt', 'die wiederkehr', regex=True)
            )
df_split.head(2)

##### df_merged

In [ ]:

df_merged = df_split.merge(linkage_df,
                           left_on='Alter Name (Deutsch)',
                           right_on='Alter Name (Deutsch)',
                           how='outer',
                           indicator=False)

df_merged = df_merged.merge(hexagramme_df, on='Nummer', how='outer', indicator=True)

#df_split = df_split['meaning'].apply(pd.Series)
#df_split = df_split['below'].apply(pd.Series)
#df_merged = df_merged[df_merged['_merge'] != 'both']

#df_merged = df_merged.iloc[:, :4].T

cols = [
    'Nummer',    
    'name',
    #'subtitle',
    #'trigrams',
    #'meaning',
    'Alter Name (Deutsch)',
    'Alter Code',
    'Neuer Code',
    'Vollständiger Name (Deutsch)',
    'Bezeichnung',
    #'Schriftzeichen',
    #'Pīnyīn',
    #'Line_Values',
    #'Title',
    'Hexagram',
    'Description',
    #'_merge'
    ]

#df_merged = df_merged.loc[:, cols].set_index('Nummer')
df_merged#.columns.to_list()
df_merged.head(2)

In [ ]:
df = df_merged.copy()

for col in df.columns:
    print(f"Spalte: {col}")
    print("Inhalt:", df.loc[0, col])
    print()

In [ ]:
import pandas as pd
import json
from dataclasses import dataclass, asdict
from typing import Dict, Optional

@dataclass
class Trigram:
    name: str
    attributes: str

@dataclass
class HexagramMeaning:
    description: str
    season: Optional[str] = None

@dataclass
class Hexagram:
    id: int
    hexagram_symbol: str
    line_values: str
    names: Dict[str, str]
    trigrams: Dict[str, Trigram]
    meaning: HexagramMeaning

    @classmethod
    def from_dataframe_row(cls, row):
        names = {
            "chinese": row["Schriftzeichen"],
            "pinyin": row["Pīnyīn"],
            "german": row["Vollständiger Name (Deutsch)"],
            "english": row["Title"],
            "old_german_name": row["Alter Name (Deutsch)"],
            "old_code": row["Alter Code"],
            "new_code": row["Neuer Code"]
        }
        
        trigrams = {
            "above": Trigram(**row["trigrams"]["above"]),
            "below": Trigram(**row["trigrams"]["below"])
        }
        
        meaning = HexagramMeaning(**row["meaning"])
        
        return cls(
            id=row["Nummer"],
            hexagram_symbol=row["Hexagram"],
            line_values=row["Line_Values"],
            names=names,
            trigrams=trigrams,
            meaning=meaning
        )

def hexagram_to_dict(hexagram: Hexagram) -> dict:
    """Convert a Hexagram object to a dictionary for JSON serialization."""
    result = asdict(hexagram)
    return result

def save_hexagrams_to_json(df: pd.DataFrame, output_file: str):
    """Convert DataFrame to structured JSON format and save to file."""
    hexagrams = []
    
    for _, row in df.iterrows():
        hexagram = Hexagram.from_dataframe_row(row)
        hexagrams.append(hexagram_to_dict(hexagram))
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(hexagrams, f, ensure_ascii=False, indent=2)

def load_hexagrams_from_json(input_file: str) -> list[Hexagram]:
    """Load hexagrams from JSON file and convert to Hexagram objects."""
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    return [Hexagram(**hexagram_data) for hexagram_data in data]



In [ ]:

# Example usage:
# Save the data
save_hexagrams_to_json(df, 'export/hexagrams_test_v1.json')

# Load the data
hexagrams = load_hexagrams_from_json('export/hexagrams_test_v1.json')

## 5. Initialisierung des Modells

### Systemanweisung für das Modell

In [ ]:
HEXAGRAM_PROMPT = """
XXX
"""

instruction = HEXAGRAM_PROMPT

In [ ]:
# Erstellen des Modells
model = genai.GenerativeModel(
    model_type,
    system_instruction=instruction
)

# Chat-Instanz starten
chat = model.start_chat()
print("Modell und Chat-Instanz erfolgreich initialisiert.")